![book header](pictures/header.png)

# Module 5: State tracking and Control

This chapter contains information that you will need when preparing for the second part: the final challenge. The ultimate goal is to let KITT drive from an initial (known) location A to a given target location B. In this final challenge you are on your own and no explicit steps are provided to help you through the process. Nevertheless, a few hints are given in this chapter on how you could attack the problem (but many alternative solution approaches exist).

**Learning objectives:** Basics of system engineering. Extension of control theory knowledge. 

**Deliverables:** 
- Sections of your final report regarding the problem analysis and overall design, and the
more detailed design of the control system.
- Python code for control that was tested on your virtual car model.

**Preparation:** Read the chapter. This module provides ideas you may use.

**What is needed:** Python code that implements your virtual car model. 

**Time duration:** One session for brainstorming (high-level system design). Two sessions for developing
the control system in Python and debugging it. Two or more sessions (evolving into system integration) for trajectory control analysis, development, and verification. Three homework sessions for study and report writing.

## Defining your approach

At the beginning of this second part, it merits to brainstorm with your entire group on the approach
that you will take. For your selected approach, determine whether it will do the job (and under what assumptions/conditions). You can opt for a simple and robust “combinatorial” approach (i.e., based on
many if-then-else statements), that will be able to handle the expected situations but not more than that,
or a more thoughtful approach based on control theory, that is much more robust but also more risky to
implement. 

This high-level system design is part of system engineering, i.e., with the entire system in mind, define
an approach at a high level that will do the job, and define specifications of each constituting subsystem.
These parts can then be designed by specialists that don’t have to know or consider the entire system. If
the parts are tested and verified to meet their specifications, then the entire system is supposed to work.
(Or so you would think.)

**Deliverable**

In your final report, document your selected approach, and the alternatives that you considered, plus your
motivation for the selection.

Document the foreseen consequences of this choice. Draw a block scheme that shows what software
blocks should interact, and define the interaction (e.g., variables).

Your solution will probably contain a finite state machine and/or a loop. An important consideration is
the timing of this loop. Document your analysis of this in sufficient detail. How often do you intend to
measure your location, keeping in mind the constraints and trade-offs here. Consider also the various
delays in the system. If you obtain a location fix, how old is that information? If you estimate velocity
from two locations, then to what time point does that velocity refer? How fast should one iteration of the
loop be? (If it is too fast, then you won’t have new location information, but if it is too slow, then you
might miss your target.) Can you merge location fixes from the audio beacon with predictions of your
position using your car model?

The text on this in your final report can be placed into an initial section “Problem definition and analysis”, or “Problem analysis and high-level design”, depending on how you want to organize your report. Alternatively, it can be placed after the localization and car modeling sections, if you need to use information from those sections.


## Defining a target path to the goal

Although not necessary, it is helpful to create a target trajectory from A to B. Assume that you know your
starting position and orientation (i.e., the initial state). From here, you can draw a feasible trajectory to the target *B* in many ways.

- You have some freedom in your orientation when you arrive at B.
- Since steering amounts to driving on circles, you can define a circle with the smallest possible
radius and then drive straight, or define a circle with a larger radius that goes through both given
points and is tangential to the given orientation, or anything in between.
- In some cases you may need to drive backwards to be able to reach the goal. E.g., what happens
if the target is within the smallest turning circle? [Note: This scenario is not part of the basic
challenges, but you could use it for the Free Challenge. See Chapter 9.(@B update)]
- Depending on your control strategy, you may need to recreate the target trajectory each time new
location information is measured.

**Deliverable** 

In the final report, document your path planning solution. Illustrate this with examples of generated routes under varying conditions. 

## What are the constrains to moving toward the goal ?

First lets consider two secnarios and then apply it to the case of KIIT: 

Imagine you have a book on a smooth, flat table. You can push this book in any direction you want: forward, backward, left, right, or any diagonal direction. You can also rotate it freely while sliding it. The book can go from its current position to any other position on the table directly, without any constraints on how it can move. This is an example of holonomic system.

Now Imagine you're trying to move a car on a flat parking lot. You can steer the car to the left or right, and you can drive forward or backward, but you can't just move the car sideways—like sliding it directly to the left or right without turning it. This limitation is an example of a non-holonomic constraint. 


Now in case of KIIT: You would like to move from point A to point B. You can't just magically place the KIIT into B sideways. Instead, you need to carefully maneuver—turn the wheel, move forward, adjust, move backward, and so on.You even have constrains reagarding how much you can steer left and right. And the path you take depends on the current orientation and motion of the car. This makes this a classic non-holonomic problem.




### Partitioning by projection

A solution to this problem is given by projection. If you could project our two-dimensional space onto
a one-dimensional space S, then you could use KITT’s position in S for one-dimensional control. Figure
8.1 (@B opdate) depicts this idea. The line represents the trajectory that you would want to follow.

Let KITT's trajectory be given by, 

$$ \mathbf{x}(t) = \left[ x(t), y(t) \right]^T $$

You can define the projection of \((x, y)\) onto \(S\) by
$$
z = z(x, y)
$$

where $z \in S$. The projected position $z$ can be regarded as the distance you travel on $ S $. KITT's speed should then be continuously controlled so that $z$ approaches the desired distance without overshooting. Thus, using $z$ as a measure for distance allows for one-dimensional control in a plane.

This approach requires knowing KITT's trajectory: you need an *estimate* of KITT's future movement. This is given by the planned trajectory. You can now state your wanted projection: The projected position $z$ is given by the arc length of the planned trajectory from KITT's current position to its target position. This allows for velocity control along any trajectory in a plane.

 <img src="pictures/trajectory.jpg" alt="KITTtraject-figure" width="250px">

*KITT's trajectory*
<!--
```{figure} trajectory.jpg
---
height/width: 150px
name: KITTtraject-figure
---
KITT's trajectory
```
-->

### Following your goal

In the previous section, you assumed that you were able to let KITT follow any predefined trajectory. You will now design a controller which keeps KITT on track.

Intuitively, you will think of a solution where you know your current position/orientation, and you always steer towards the target. Once you are oriented towards the target, your "angle error" is zero, and you just have to drive straight. In all other cases, the angle error determines how much you need to steer. It is easy to see that this approach might work, but also might become unstable once you are very close to the target.

Suppose KITT is driving on its trajectory $S$, where its orientation with respect to the $x$-axis is given by $\theta$. Let the angle tangent to its trajectory $S$ be given by $\theta_t$. Ideally, this angle should be equal to KITT's orientation. If KITT's orientation deviates from this angle, KITT should turn its wheels to steer back. This motivates a feedback law given by

$$
\phi = -k(\theta - \theta_t)
$$

for a positive $k$. Substituting in Equation (\ref{eq:steering-derivative}) (@B update) yields the autonomous non-linear system

$$
L \,\frac{d}{dt}\theta \;+\; v \sin(k(\theta-\theta_t)) = 0\,.
$$

You should choose $k$ such that this system is asymptotically stable. To investigate the stability, you introduce a potential function (error function) given by

$$
T(\theta) = \frac{1}{2}(\theta-\theta_t)^2 \,.
$$

A first observation is that $T(\theta)=0$ if and only if $\theta=\theta_t$, which is exactly what you want. Second, notice that $T(\theta) \ge 0$. you can conclude that $T$'s minimum corresponds to our equilibrium point. Notice that

$$
\frac{d}{dt} T(\theta) = -(\theta-\theta_t)\frac{v \sin{(k(\theta-\theta_t))}}{L}.
$$

Figure \ref{fig:KITTpotential-figure}  (@B update) depicts both \(T(\theta)\) and its time-derivative.



<img src="pictures/potentialfunction.jpg" alt="KITTpotential-figure" width="250px">

*Graph of potential function and it's derivative*
<!-- 
```{figure} potentialfunction.jpg
---
height/width: 150px
name: KITTpotential-figure
---
Graph of potential function and it's derivative
```
-->

Consider KITT's orientation at any instant. If $ |\theta| < \theta_m $, then by Figure \ref{fig:lyaponuv} the potential function will have a negative derivative. But then it will decrease over time and will keep decreasing until $\theta = \theta_t$. So in conclusion, if $ |\theta| < \theta_m $, then $\theta$ will converge to its equilibrium point. By inspection of Equation (\ref{eq:lyapunov}) you can now state that Equation (\ref{eq:angle-diffeq}) is locally asymptotically stable for any

$$
-\frac{\pi}{k} < \theta - \theta_t < \frac{\pi}{k}.
$$

This treatment is also called the investigation of *Lyapunov stability*, where $T$ is called the *Lyapunov function*. It is an extensive topic in the control literature.

### Deliverable

Implement your control algorithm and test it using your virtual car model. In your final report, document the results of the tests, which prove that given correct position estimation, the control algorithm will get the car to its goal.

A possible implementation could be as teh code below :

The `challenge_A_KITT()` function is responsible for aligning KITT's trajectory to point B and driving it to reach point B using the minimum distance. 

1. **Initial Check**: The function first checks if KITT is "almost there" (line 6). If true, it calculates the distance between KITT's current position and point B (`accuracy`) and the minimum distance from KITT's trajectory to point B (`min_distance`).

2. **Positioning at Point B**: If the accuracy is within a threshold (less than or equal to 20 cm), KITT stops and marks the mission as accomplished (lines 9-15). It prints the current position and accuracy, indicating that point B has been located.

3. **Speed Control**: If KITT is not yet at point B, it adjusts its speed depending on the battery voltage, using higher speed when the voltage is low to conserve power (lines 19-26 and 32-39).

4. **Trajectory Alignment**: If KITT is on the route to point B (line 28), the system continues moving while adjusting the angle and speed. KITT tries to steer towards point B by calculating the error between the current and desired orientation and applying a proportional-derivative (PD) control to determine the steering angle (lines 65-74).

5. **Steering Adjustments**: Based on the calculated steering angle, KITT adjusts its steering to either correct its trajectory or handle small deviations in its path (lines 76-97). The steering command is adjusted in degrees depending on how far KITT is from the correct trajectory.

6. **Boundary Handling**: If KITT approaches the grid boundaries (too close to edges), it sets a safe speed and adjusts its angle to avoid going out of bounds (lines 98-105).

7. **Final Position Update**: Once the vehicle reaches point B, or if obstacles are detected en route, the system stops and updates KITT's position. If the calculated trajectory ensures that KITT is still on course to point B (line 130), the mission proceeds as planned.

The function also manages smaller details like ensuring obstacle avoidance and optimizing movement based on current battery levels throughout the process.

In [ ]:
def challenge_A_KITT():
    
    global mission_A_accomplished, accuracy

    if kittmodel.almost_there == 1:
        # Calculate the accuracy and minimum distance to point B
        accuracy = np.sqrt((kittmodel.dist_x - B_pos[0]) ** 2 + (kittmodel.dist_y - B_pos[1]) ** 2)
        min_distance = abs(-kittmodel.slope_t1 * B_pos[0] + B_pos[1] - kittmodel.b_t1) / np.sqrt(kittmodel.slope_t1 ** 2 + 1)

        # Check if KITT has reached the required accuracy at point B
        if round(100 * accuracy) <= 20:
            print("Position B located!")
            print("Accuracy:", round(100 * accuracy, 2), "cm")
            print("x:", round(kittmodel.dist_x, 3), "y:", round(kittmodel.dist_y, 3))
            kittmodel.stop()
            kitt.stop(0)
            mission_A_accomplished = 1
            return mission_A_accomplished
        else:
            kittmodel.set_speed(158)
            if kitt.battery_voltage <= 18:
                kitt.set_speed(164)
                time.sleep(0.05)
                kitt.set_speed(158)
            else:
                kitt.set_speed(161)
                time.sleep(0.05)
                kitt.set_speed(158)

    elif kittmodel.on_route_AB == 1:
        print("Found trajectory A-B")
        kittmodel.set_speed(158)
        kitt.set_angle(150)
        
        if kitt.battery_voltage <= 18:
            kitt.set_speed(164)
            time.sleep(0.05)
            kitt.set_speed(158)
        else:
            kitt.set_speed(161)
            time.sleep(0.05)
            kitt.set_speed(158)

        # Recalculate accuracy and minimum distance
        accuracy = np.sqrt((kittmodel.dist_x - B_pos[0]) ** 2 + (kittmodel.dist_y - B_pos[1]) ** 2)
        min_distance = abs(-kittmodel.slope_t1 * B_pos[0] + B_pos[1] - kittmodel.b_t1) / np.sqrt(kittmodel.slope_t1 ** 2 + 1)

        if init_TDOA == 1:
            if round(100 * accuracy) <= 25:
                print("In range 25 cm")
                kitt.stop(0)
                kittmodel.stop()
                kitt.read_microfoon()
                kittmodel.almost_there = 1
            else:
                if round(100 * accuracy) <= 10:
                    print("Position B located!")
                    print("Accuracy:", round(100 * accuracy, 2), "cm")
                    print("x:", round(kittmodel.dist_x, 3), "y:", round(kittmodel.dist_y, 3))
                    kittmodel.stop()
                    kitt.stop(0)
                    mission_A_accomplished = 1
                    return mission_A_accomplished
                else:
                    # PID controller for steering adjustment
                    Kp = 3
                    Kd = 0.1

                    current_orientation = np.arctan2(kittmodel.d_vector_0[1], kittmodel.d_vector_0[0])
                    desired_orientation = np.arctan2(B_pos[1] - kittmodel.dist_y, B_pos[0] - kittmodel.dist_x)

                    error = desired_orientation - current_orientation
                    error = (error + np.pi) % (2 * np.pi) - np.pi
                    derivative = (error - kittmodel.prev_error) / kittmodel.delta_t
                    kittmodel.prev_error = error

                    steering_angle = Kp * error + Kd * derivative

                    # Steering command based on the calculated steering angle
                    if steering_angle < 0:
                        if abs(steering_angle) <= np.radians(1):
                            steering_command = 150
                        elif abs(steering_angle) <= np.radians(5):
                            steering_command = 185
                        elif abs(steering_angle) <= np.radians(10):
                            steering_command = 190
                        elif abs(steering_angle) <= np.radians(15):
                            steering_command = 195
                        else:
                            steering_command = 200
                    else:
                        if abs(steering_angle) <= np.radians(1):
                            steering_command = 150
                        elif abs(steering_angle) <= np.radians(5):
                            steering_command = 115
                        elif abs(steering_angle) <= np.radians(10):
                            steering_command = 110
                        elif abs(steering_angle) <= np.radians(15):
                            steering_command = 105
                        else:
                            steering_command = 100

                    # Boundary condition check
                    if (kittmodel.dist_x >= 4.6 or kittmodel.dist_y >= 4.6 or kittmodel.dist_x <= 0.2 or kittmodel.dist_y <= 0.2):
                        kitt.set_angle(150)
                        kitt.set_speed(142)
                        kitt.set_speed(139)
                        time.sleep(0.1)
                        kitt.set_speed(142)
                        time.sleep(1.4)
                        kitt.stop(1)
                        kitt.read_microfoon()
                    else:
                        kittmodel.set_angle(steering_command)
                        kitt.set_angle(steering_command)
                        if steering_command == 150:
                            kittmodel.set_speed(158)
                        else:
                            kittmodel.set_speed(160)
                        if kitt.battery_voltage <= 18.1:
                            kitt.set_speed(163)
                            time.sleep(0.1)
                            kitt.set_speed(158)
                        else:
                            kitt.set_speed(161)
                            time.sleep(0.1)
                            kitt.set_speed(158)

        # Final position check with discriminant for correct trajectory
        k = 1 + kittmodel.slope_t1 ** 2
        l = 2 * kittmodel.slope_t1 * (kittmodel.b_t1 - B_pos[1]) - 2 * B_pos[0]
        m = B_pos[0] ** 2 + (kittmodel.b_t1 - B_pos[1]) ** 2 - 0.12 ** 2

        discriminant_2A = l ** 2 - 4 * k * m

        if (discriminant_2A >= 0 and ((B_pos[0] - kittmodel.dist_x) * kittmodel.d_vector_0[0] >= 0 or
                                      (B_pos[1] - kittmodel.dist_y) * kittmodel.d_vector_0[1] >= 0)):
            kittmodel.stop()
            kitt.stop(0)
            kittmodel.on_route_AB = 1


## Obstacle avoidance

Obstacle avoidance is an advanced topic and you will only get to this if you got trajectory tracking completely solved and working.

You have the parking sensors to help you detect obstacles, and also the perimeter of the field can be considered an (invisible) obstacle. Once you detect an obstacle, you need to steer around it. Some suggested approaches are:

- Define a new planned trajectory around the obstacle (possibly requiring driving backwards). Typically, students follow a *combinatorial approach*: lots of if-then-else statements. This works in simple cases. The disadvantage is that it is hard to debug, and the solutions are often not general.
- Study control literature on obstacle avoidance that use *artificial potential fields*. This is a general approach that essentially defines a penalty function around obstacles and then finds optimal trajectories that minimize the "cost".




Simple soultion for obstacle avoidance:

The code below implements obstacle detection and avoidance for KITT to complete challenges C and D. It begins by reading data from KITT’s ultrasonic sensors, which detect the distance to obstacles around the car. The function `get_distance_update()` requests distance data and decodes it to extract the distance to obstacles on the left and right sides of the car. The data is updated with an offset of 4%, as specified in the challenge. 

If the distance to any obstacle is less than or equal to 50 cm, KITT stops. This ensures that it maintains a safe distance from obstacles, matching the requirement from the challenge to avoid collisions with a 50 cm buffer for safety. The position of the obstacle is calculated and printed for reference, allowing KITT to document its location. The car then evaluates whether it should adjust its movement depending on whether the obstacle is on the left or right side, or directly in front.

 KITT handles situations where it approaches the grid boundaries as if they were obstacles. When it detects that it's near a boundary, KITT stops, adjusts its angle, and backs away to avoid going out of bounds, following the same maneuver as it does with physical obstacles.



In [ ]:

def get_distance_update():
    
    kitt.serial.write(b'Sd\n')
    distance = kitt.serial.read_until(b'\x04')
    distance = distance.decode('utf-8')
    
    dist_left = int(distance[int(distance.find('L') + 1):int(distance.find('R') - 3)])
    dist_right = int(distance[int(distance.find('R') + 1):int(distance.find('x') - 1)])
    
    offset = 4  # %, as measured
    
    # Use the parking sensors, Obstacle Detection, Threshold of 50 cm or the boundaries of the grid
    if (kittmodel.dist_x >= 4.4 or kittmodel.dist_y >= 4.4 or kittmodel.dist_x <= 0.2 or kittmodel.dist_y <= 0.2 or dist_left <= 60 or dist_right <= 60):
        if dist_left <= 60 and dist_right <= 60:
            kitt.Obstacle_pos = (kittmodel.dist_x + kittmodel.d_vector_0[0] * ((dist_left / 100) * (100 - offset) + 0.21),
                                 kittmodel.dist_y + kittmodel.d_vector_0[1] * (dist_left / 100 + 0.21))
            print("Obstacle Detected in front of the Car at Location:", kitt.Obstacle_pos)
        
        if dist_left <= 60:
            kitt.Obstacle_pos = (kittmodel.dist_x + kittmodel.d_vector_0[0] * ((dist_left / 100) * (100 - offset) + 0.21),
                                 kittmodel.dist_y + kittmodel.d_vector_0[1] * (dist_left / 100 + 0.21))
            print("Obstacle Detected left of the Car at", kitt.Obstacle_pos)
        
        if dist_right <= 60:
            kitt.Obstacle_pos = (kittmodel.dist_x + kittmodel.d_vector_0[0] * ((dist_right / 100) * (100 - offset) + 0.21),
                                 kittmodel.dist_y + kittmodel.d_vector_0[1] * (dist_right / 100 + 0.21))
            print("Obstacle Detected right of the Car at", kitt.Obstacle_pos)
        
        kitt.stop(0)
        return 1
    else:
        return 0

obstacle_detected = get_distance_update()
kittmodel.obstacle_trajectory = kittmodel.d_vector_0

if kittmodel.obstacle_seen >= 1 and kittmodel.obstacle_seen <= 8:
    obstacle_detected = 0

# Obstacle avoidance
kittmodel.obstacle_seen += 1
kitt.set_angle(kittmodel.avoidance_turn)
kittmodel.set_angle(kittmodel.avoidance_turn)
kittmodel.shifting(158)

if kitt.battery_voltage <= 18:
    kitt.set_speed(164)
    time.sleep(0.05)
    kitt.set_speed(158)
else:
    kitt.set_speed(161)
    time.sleep(0.05)
    kitt.set_speed(158)

if obstacle_detected == 1:
    kittmodel.on_route_AB = 0
    if kitt.battery_voltage <= 18:
        kitt.set_angle(150)
        kitt.set_speed(135)
        time.sleep(0.1)
        kitt.set_speed(142)
        time.sleep(1.4)
    else:
        kitt.set_angle(150)
        kitt.set_speed(139)
        time.sleep(0.1)
        kitt.set_speed(142)
        time.sleep(1.4)
    
    kitt.stop(1)
    
    if init_TDOA == 1:
        kitt.read_microfoon()
    else:
        kittmodel.dist_x -= kittmodel.d_vector_0[0] * 0.5
        kittmodel.dist_y -= kittmodel.d_vector_0[1] * 0.5
        kittmodel.obstacle_seen += 1

product1 = kittmodel.dist_x * (4.6 - kittmodel.dist_y)
product2 = (4.6 - kittmodel.dist_x) * kittmodel.dist_y
product3 = kittmodel.dist_x * kittmodel.dist_y
product4 = (4.6 - kittmodel.dist_x) * (4.6 - kittmodel.dist_y)

if kittmodel.obstacle_trajectory[0] >= 0 and kittmodel.obstacle_trajectory[1] >= 0:
    kittmodel.avoidance_turn = 100 if product1 >= product2 else 200
elif kittmodel.obstacle_trajectory[0] <= 0 and kittmodel.obstacle_trajectory[1] >= 0:
    kittmodel.avoidance_turn = 200 if product3 >= product4 else 100
elif kittmodel.obstacle_trajectory[0] >= 0 and kittmodel.obstacle_trajectory[1] <= 0:
    kittmodel.avoidance_turn = 200 if product3 >= product4 else 100
else:
    kittmodel.avoidance_turn = 200 if product1 >= product2 else 100
